In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dpl-302-m-ai-1810-assignment-2/train data.json
/kaggle/input/dpl-302-m-ai-1810-assignment-2/test.csv
/kaggle/input/datata/train data.json
/kaggle/input/datata/test.csv


In [5]:
"""
Phân loại tình cảm văn bản tiếng Việt sử dụng BiLSTM
Tối ưu cho tác vụ phân loại tình cảm (sentiment classification)
"""

import os
import re
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import unicodedata  # Thêm thư viện xử lý Unicode

# Cấu hình
MAX_LENGTH = 100
VOCAB_SIZE = 20000
EMBEDDING_DIM = 128
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.001  # Khôi phục learning rate ban đầu

class SentimentAttention(tf.keras.layers.Layer):
    """Lớp Attention tùy chỉnh cho phân loại cảm xúc"""
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(SentimentAttention, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Đảm bảo input_shape có thể là 2D hoặc 3D
        feature_dim = input_shape[-1]
        
        # Tạo ma trận trọng số và bias
        self.W = self.add_weight(
            name="attention_weight",
            shape=(feature_dim, 1),
            initializer="glorot_uniform"
        )
        self.b = self.add_weight(
            name="attention_bias",
            shape=(1,),
            initializer="zeros"
        )
        self.built = True
        
    def call(self, inputs, mask=None):
        # Kiểm tra kích thước của inputs
        input_shape = tf.shape(inputs)
        
        # Nếu chỉ có 2 chiều (batch_size, features) thì áp dụng attention trực tiếp
        if len(inputs.shape) == 2:
            # Khi input là từ dense layer, áp dụng dot product một lần
            logits = tf.matmul(inputs, self.W) + self.b  # shape=(batch_size, 1)
            attention_weights = tf.nn.sigmoid(logits)  # Sử dụng sigmoid thay vì softmax cho 1 phần tử
            
            # Tạo output = input (không cần weighted sum vì chỉ có 1 vector đặc trưng)
            output = inputs
            
            return output, tf.squeeze(attention_weights, axis=-1)
        
        # Xử lý cho tensor 3D (batch_size, timesteps, features)
        else:
            # Áp dụng tanh(Wx + b) cho mỗi timestep
            uit = tf.tanh(tf.matmul(
                tf.reshape(inputs, [-1, input_shape[-1]]),  # Reshape để dot product
                self.W
            ) + self.b)
            
            # Reshape về (batch_size, timesteps, 1)
            uit = tf.reshape(uit, [-1, input_shape[1], 1])
            
            # Áp dụng mask nếu có
            if mask is not None:
                mask = tf.cast(mask, tf.bool)
                mask = tf.expand_dims(mask, axis=-1)  # (batch_size, timesteps, 1)
                paddings = tf.ones_like(uit) * (-1e9)
                uit = tf.where(mask, uit, paddings)
            
            # Tính softmax theo chiều timesteps
            attention_weights = tf.nn.softmax(uit, axis=1)  # (batch_size, timesteps, 1)
            
            # Áp dụng attention weights cho inputs
            weighted_input = inputs * attention_weights
            
            # Tổng hợp theo chiều timesteps
            output = tf.reduce_sum(weighted_input, axis=1)  # (batch_size, features)
            
            return output, tf.squeeze(attention_weights, axis=-1)
    
    def compute_output_shape(self, input_shape):
        # Output là (batch_size, features) và (batch_size, sequence_length)
        if len(input_shape) == 3:
            return [(input_shape[0], input_shape[2]), (input_shape[0], input_shape[1])]
        else:  # Trường hợp 2D
            return [(input_shape[0], input_shape[1]), (input_shape[0], 1)]
    
    def get_config(self):
        config = super(SentimentAttention, self).get_config()
        return config

class VietnameseTextClassifier:
    def __init__(self):
        print("Khởi tạo Vietnamese Text Classifier với mô hình BiLSTM...")
        self.tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
        self.topic_threshold = 0.2  # Giảm ngưỡng chủ đề từ 0.5 xuống 0.2
        
    def load_train_data(self, json_path):
        """Tải dữ liệu huấn luyện từ file JSON (hỗ trợ định dạng Label Studio)"""
        print(f"Đang tải dữ liệu huấn luyện từ {json_path}...")
        
        try:
            # Kiểm tra file tồn tại
            if not os.path.exists(json_path):
                print(f"Lỗi: Không tìm thấy file {json_path}")
                return pd.DataFrame({'text': [], 'sentiment': [], 'topics': []})
            
            # Tải dữ liệu JSON
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # In thông tin debug
            print(f"Cấu trúc dữ liệu: {type(data)}")
            if isinstance(data, list):
                print(f"Số lượng mục: {len(data)}")
            
            # Xử lý dữ liệu
            rows = []
            
            # Xử lý định dạng Label Studio
            if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict) and 'annotations' in data[0]:
                print("Phát hiện định dạng Label Studio")
                
                for item in data:
                    # Trích xuất văn bản
                    text = ""
                    if 'data' in item and isinstance(item['data'], dict):
                        for field in ['text', 'content', 'document']:
                            if field in item['data'] and isinstance(item['data'][field], str):
                                text = item['data'][field].strip()
                                break
                    
                    if not text:
                        continue
                    
                    # Trích xuất tình cảm và chủ đề
                    sentiment = "Trung tính"  # Mặc định
                    topics = []  # Danh sách chủ đề
                    
                    if 'annotations' in item and isinstance(item['annotations'], list):
                        for annotation in item['annotations']:
                            if 'value' in annotation and isinstance(annotation['value'], dict):
                                # Tìm tình cảm trong choices
                                if 'choices' in annotation['value'] and isinstance(annotation['value']['choices'], list):
                                    for choice in annotation['value']['choices']:
                                        if choice in ["Tích cực", "Tiêu cực", "Trung tính"]:
                                            sentiment = choice
                                        else:
                                            # Nếu không phải tình cảm, có thể là chủ đề
                                            topics.append(choice)
                                
                                # Tìm chủ đề trong labels
                                if 'labels' in annotation['value'] and isinstance(annotation['value']['labels'], list):
                                    topics.extend(annotation['value']['labels'])
                    
                    # Thêm vào danh sách
                    rows.append({
                        'text': self.clean_text(text),
                        'sentiment': sentiment,
                        'topics': topics
                    })
            else:
                # Xử lý định dạng thông thường
                for item in data if isinstance(data, list) else [data]:
                    if isinstance(item, dict) and 'text' in item:
                        topics = []
                        if 'topics' in item and isinstance(item['topics'], list):
                            topics = item['topics']
                        elif 'labels' in item and isinstance(item['labels'], list):
                            topics = item['labels']
                        
                        rows.append({
                            'text': self.clean_text(item['text']),
                            'sentiment': item.get('sentiment', "Trung tính"),
                            'topics': topics
                        })
            
            # Tạo DataFrame
            df = pd.DataFrame(rows)
            print(f"Đã tải {len(df)} mẫu dữ liệu huấn luyện")
            
            # Kiểm tra chủ đề
            if 'topics' in df.columns:
                all_topics = set()
                for topics_list in df['topics']:
                    if isinstance(topics_list, list):
                        all_topics.update(topics_list)
                print(f"Tìm thấy {len(all_topics)} chủ đề khác nhau")
            
            return df
            
        except Exception as e:
            print(f"Lỗi khi tải dữ liệu huấn luyện: {str(e)}")
            return pd.DataFrame({'text': [], 'sentiment': [], 'topics': []})
    
    def clean_text(self, text):
        """Làm sạch văn bản tiếng Việt với các cải tiến cho phân loại cảm xúc"""
        if not isinstance(text, str):
            return ""
        
        # Chuẩn hóa Unicode (NFC)
        text = unicodedata.normalize('NFC', text)
        
        # Thay thế emojis bằng mô tả
        text = self._replace_emojis(text)
        
        # Xử lý kí tự lặp (ví dụ: "quáaaaaa" -> "quá")
        text = re.sub(r'([A-Za-z])\1{2,}', r'\1', text)
        
        # Chuẩn hóa dấu câu quan trọng cho cảm xúc
        text = re.sub(r'!{2,}', ' ! ! ', text)  # Chuẩn hóa nhiều dấu chấm than
        text = re.sub(r'\?{2,}', ' ? ? ', text)  # Chuẩn hóa nhiều dấu hỏi
        
        # Thay thế nhiều khoảng trắng bằng một khoảng trắng
        text = re.sub(r'\s+', ' ', text)
        # Loại bỏ URL và thẻ HTML
        text = re.sub(r'https?://\S+|www\.\S+|<.*?>', '', text)
        # Loại bỏ khoảng trắng thừa
        return text.strip()
    
    def _replace_emojis(self, text):
        """Thay thế emojis phổ biến bằng từ mô tả cảm xúc"""
        emoji_map = {
            '😊': ' vui_mừng ',
            '😄': ' cười_lớn ',
            '😃': ' vui_vẻ ',
            '😀': ' cười ',
            '😍': ' yêu_thích ',
            '🥰': ' rất_yêu_thích ',
            '😘': ' hôn_gió ',
            '❤️': ' trái_tim ',
            '♥️': ' yêu ',
            '👍': ' thích ',
            '👎': ' không_thích ',
            '😢': ' buồn ',
            '😭': ' khóc ',
            '😡': ' tức_giận ',
            '🤬': ' rất_tức_giận ',
            '🤔': ' suy_nghĩ ',
            '😱': ' sợ_hãi ',
            '🙄': ' ngán_ngẩm ',
            '🤣': ' cười_lăn ',
            '😂': ' cười_ra_nước_mắt '
        }
        
        for emoji, replacement in emoji_map.items():
            text = text.replace(emoji, replacement)
        
        return text
    
    def prepare_data(self, train_df):
        """Chuẩn bị dữ liệu cho huấn luyện và kiểm tra (sử dụng split từ dữ liệu JSON)"""
        if len(train_df) == 0:
            print("Cảnh báo: DataFrame huấn luyện rỗng, không thể chuẩn bị dữ liệu")
            return None, None
        
        # Tạo ánh xạ nhãn tình cảm
        unique_sentiments = sorted(list(set(train_df['sentiment'])))
        sentiment_to_idx = {sentiment: i for i, sentiment in enumerate(unique_sentiments)}
        
        # Tạo ánh xạ nhãn chủ đề
        all_topics = set()
        if 'topics' in train_df.columns:
            for topics_list in train_df['topics']:
                if isinstance(topics_list, list):
                    all_topics.update(topics_list)
        
        unique_topics = sorted(list(all_topics))
        topic_to_idx = {topic: i for i, topic in enumerate(unique_topics)}
        
        # Cân bằng dữ liệu sentiment trước khi chia
        # Đếm số lượng mẫu cho mỗi nhãn cảm xúc
        sentiment_counts = train_df['sentiment'].value_counts()
        max_samples = sentiment_counts.max()
        
        # Tăng cường dữ liệu cho các nhóm thiểu số
        augmented_rows = []
        for sentiment, count in sentiment_counts.items():
            if count < max_samples:
                # Lấy tất cả các mẫu của sentiment này
                sentiment_samples = train_df[train_df['sentiment'] == sentiment]
                # Số lượng mẫu cần tạo thêm
                num_to_generate = max_samples - count
                
                # Tạo thêm mẫu bằng cách nhân bản và thêm biến thể nhẹ
                for i in range(num_to_generate):
                    # Chọn ngẫu nhiên một mẫu để nhân bản
                    sample = sentiment_samples.sample(1).iloc[0]
                    
                    # Tạo biến thể nhẹ cho text
                    original_text = sample['text']
                    augmented_text = self._augment_text(original_text)
                    
                    # Tạo mẫu mới
                    augmented_row = sample.copy()
                    augmented_row['text'] = augmented_text
                    
                    augmented_rows.append(augmented_row)
        
        # Thêm các mẫu tăng cường vào DataFrame
        if augmented_rows:
            augmented_df = pd.DataFrame(augmented_rows)
            train_df = pd.concat([train_df, augmented_df], ignore_index=True)
            print(f"Đã tạo thêm {len(augmented_rows)} mẫu tăng cường để cân bằng dữ liệu cảm xúc")
        
        # Chia dữ liệu huấn luyện và validation
        train_split_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
        print(f"Chia dữ liệu: {len(train_split_df)} mẫu huấn luyện, {len(val_df)} mẫu validation")
        
        # Huấn luyện tokenizer trên tập huấn luyện
        self.tokenizer.fit_on_texts(train_split_df['text'])
        
        # Chuyển đổi văn bản thành chuỗi số
        train_sequences = self.tokenizer.texts_to_sequences(train_split_df['text'])
        val_sequences = self.tokenizer.texts_to_sequences(val_df['text'])
        
        # Đệm chuỗi để có cùng độ dài
        train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post')
        val_padded = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post')
        
        # Chuẩn bị nhãn tình cảm
        train_sentiment_labels = self._prepare_sentiment_labels(train_split_df['sentiment'].tolist(), sentiment_to_idx)
        val_sentiment_labels = self._prepare_sentiment_labels(val_df['sentiment'].tolist(), sentiment_to_idx)
        
        # Chuẩn bị nhãn chủ đề (multi-label)
        train_topic_labels = self._prepare_topic_labels(train_split_df['topics'].tolist(), topic_to_idx)
        val_topic_labels = self._prepare_topic_labels(val_df['topics'].tolist(), topic_to_idx)
        
        # Tạo dictionary dữ liệu
        data = {
            'train': {
                'sequences': train_padded,
                'sentiment_labels': train_sentiment_labels,
                'topic_labels': train_topic_labels
            },
            'val': {
                'sequences': val_padded,
                'sentiment_labels': val_sentiment_labels,
                'topic_labels': val_topic_labels
            }
        }
        
        # Tạo ánh xạ nhãn và từ vựng
        mappings = {
            'unique_sentiments': unique_sentiments,
            'sentiment_to_idx': sentiment_to_idx,
            'idx_to_sentiment': {i: sentiment for sentiment, i in sentiment_to_idx.items()},
            'num_sentiments': len(sentiment_to_idx),
            'unique_topics': unique_topics,
            'topic_to_idx': topic_to_idx,
            'idx_to_topic': {i: topic for topic, i in topic_to_idx.items()},
            'num_topics': len(topic_to_idx),
            'word_index': self.tokenizer.word_index,
            'vocab_size': min(VOCAB_SIZE, len(self.tokenizer.word_index) + 1)
        }
        
        return data, mappings
    
    def _augment_text(self, text):
        """Tạo biến thể nhẹ của văn bản để tăng cường dữ liệu"""
        if not isinstance(text, str) or len(text) < 10:
            return text
            
        augmented = text
        
        # Danh sách các phép biến đổi
        transforms = [
            self._add_emphasis,
            self._swap_words,
            self._synonym_replacement,
            self._remove_random_words,
        ]
        
        # Chọn ngẫu nhiên 1-2 phép biến đổi
        num_transforms = np.random.randint(1, 3)
        selected_transforms = np.random.choice(transforms, num_transforms, replace=False)
        
        # Áp dụng các phép biến đổi
        for transform in selected_transforms:
            augmented = transform(augmented)
            
        return augmented
    
    def _add_emphasis(self, text):
        """Thêm dấu câu nhấn mạnh cảm xúc"""
        # Một số từ quan trọng và emoji tương ứng
        emphasis = [
            ("!", "!!"),
            ("?", "??"),
            ("rất", "rất rất"),
            ("tuyệt", "tuyệt vời"),
            ("tốt", "rất tốt"),
            ("không tốt", "không tốt chút nào"),
            ("tệ", "quá tệ"),
        ]
        
        result = text
        # Chọn ngẫu nhiên 1 cách nhấn mạnh để áp dụng
        choice = np.random.choice(len(emphasis))
        original, replacement = emphasis[choice]
        
        # Thay thế với xác suất 0.7
        if original in result and np.random.random() < 0.7:
            result = result.replace(original, replacement, 1)
            
        return result
    
    def _swap_words(self, text):
        """Hoán đổi vị trí của hai từ cạnh nhau"""
        words = text.split()
        if len(words) < 4:
            return text
            
        # Chọn ngẫu nhiên vị trí để hoán đổi, tránh từ đầu và cuối
        idx = np.random.randint(1, len(words) - 2)
        
        # Hoán đổi từ
        words[idx], words[idx + 1] = words[idx + 1], words[idx]
        
        return " ".join(words)
    
    def _synonym_replacement(self, text):
        """Thay thế từ bằng từ đồng nghĩa"""
        # Một số từ đồng nghĩa cơ bản cho tình cảm
        synonyms = {
            "tốt": ["hay", "tuyệt", "tích cực", "khá"],
            "tuyệt": ["tuyệt vời", "xuất sắc", "hoàn hảo"],
            "tệ": ["kém", "tồi", "không tốt", "dở"],
            "thích": ["yêu thích", "ưa", "hài lòng"],
            "buồn": ["không vui", "thất vọng", "chán nản"],
            "vui": ["vui vẻ", "hạnh phúc", "phấn khởi"]
        }
        
        words = text.split()
        for i, word in enumerate(words):
            if word in synonyms and np.random.random() < 0.3:
                # Thay thế bằng từ đồng nghĩa ngẫu nhiên
                replacement = np.random.choice(synonyms[word])
                words[i] = replacement
                break  # Chỉ thay thế một từ
                
        return " ".join(words)
    
    def _remove_random_words(self, text):
        """Loại bỏ một từ ngẫu nhiên"""
        words = text.split()
        if len(words) < 5:
            return text
            
        # Loại bỏ một từ ngẫu nhiên không quan trọng (tránh từ đầu tiên)
        idx = np.random.randint(1, len(words))
        words.pop(idx)
        
        return " ".join(words)
    
    def _prepare_sentiment_labels(self, sentiments, sentiment_to_idx):
        """Chuẩn bị nhãn tình cảm (single-label)"""
        labels = []
        for sentiment in sentiments:
            if sentiment in sentiment_to_idx:
                idx = sentiment_to_idx[sentiment]
            else:
                # Nếu không tìm thấy, sử dụng Trung tính (hoặc chỉ số thích hợp)
                idx = sentiment_to_idx.get("Trung tính", 0)
            labels.append(idx)
        
        return tf.keras.utils.to_categorical(labels, num_classes=len(sentiment_to_idx))
    
    def _prepare_topic_labels(self, topics_list, topic_to_idx):
        """Chuẩn bị nhãn chủ đề (multi-label)"""
        num_topics = len(topic_to_idx)
        labels = np.zeros((len(topics_list), num_topics))
        
        for i, topics in enumerate(topics_list):
            if isinstance(topics, list):
                for topic in topics:
                    if topic in topic_to_idx:
                        labels[i, topic_to_idx[topic]] = 1
        
        return labels
    
    def build_model(self, mappings):
        """Xây dựng mô hình BiLSTM cho phân loại tình cảm và chủ đề"""
        print("Xây dựng mô hình BiLSTM kép cho phân loại tình cảm và chủ đề...")
        
        # Lấy kích thước từ vựng và số lớp
        vocab_size = mappings['vocab_size']
        num_sentiments = mappings['num_sentiments']
        num_topics = mappings['num_topics']
        
        # Đầu vào 
        input_layer = tf.keras.layers.Input(shape=(MAX_LENGTH,), name='input_sequences')
        
        # Embedding layer
        embedding = tf.keras.layers.Embedding(
            vocab_size, EMBEDDING_DIM, input_length=MAX_LENGTH
        )(input_layer)
        
        # Shared BiLSTM layers
        lstm1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(128, return_sequences=True)
        )(embedding)
        lstm1_dropout = tf.keras.layers.Dropout(0.3)(lstm1)
        
        lstm2 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True)
        )(lstm1_dropout)
        lstm2_dropout = tf.keras.layers.Dropout(0.3)(lstm2)
        
        lstm3 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(32)
        )(lstm2_dropout)
        lstm3_dropout = tf.keras.layers.Dropout(0.3)(lstm3)
        
        # Shared Dense layers
        shared_dense1 = tf.keras.layers.Dense(256, activation='relu')(lstm3_dropout)
        shared_dense1_dropout = tf.keras.layers.Dropout(0.3)(shared_dense1)
        
        shared_dense2 = tf.keras.layers.Dense(128, activation='relu')(shared_dense1_dropout)
        shared_dense2_dropout = tf.keras.layers.Dropout(0.3)(shared_dense2)
        
        # Sentiment specific layers
        sentiment_attention = SentimentAttention()(shared_dense2_dropout)
        sentiment_dense = tf.keras.layers.Dense(64, activation='relu')(sentiment_attention[0])
        sentiment_dropout = tf.keras.layers.Dropout(0.1)(sentiment_dense)
        sentiment_output = tf.keras.layers.Dense(
            num_sentiments, activation='softmax', name='sentiment_output'
        )(sentiment_dropout)
        
        # Topic specific layers - thêm nhiều lớp dense hơn cho tác vụ multi-label phức tạp
        topic_dense1 = tf.keras.layers.Dense(128, activation='relu')(shared_dense2_dropout)
        topic_dropout1 = tf.keras.layers.Dropout(0.2)(topic_dense1)
        
        topic_dense2 = tf.keras.layers.Dense(64, activation='relu')(topic_dropout1)
        topic_dropout2 = tf.keras.layers.Dropout(0.2)(topic_dense2)
        
        topic_output = tf.keras.layers.Dense(
            num_topics, activation='sigmoid', name='topic_output'
        )(topic_dropout2)
        
        # Tạo mô hình
        model = tf.keras.Model(
            inputs=input_layer,
            outputs=[sentiment_output, topic_output]
        )
        
        return model
    
    def train(self, data, mappings):
        """Huấn luyện mô hình"""
        print("Huấn luyện mô hình...")
        
        # Tạo mô hình
        model = self.build_model(mappings)
        
        # Tính toán class weights cho sentiment
        unique_sentiment_counts = Counter(np.argmax(data['train']['sentiment_labels'], axis=1))
        total_sentiment_samples = sum(unique_sentiment_counts.values())
        sentiment_class_weights = {
            class_idx: total_sentiment_samples / (count * len(unique_sentiment_counts))
            for class_idx, count in unique_sentiment_counts.items()
        }
        
        print("Tỷ lệ cảm xúc:")
        for class_idx, weight in sentiment_class_weights.items():
            sentiment_name = mappings['idx_to_sentiment'][class_idx]
            count = unique_sentiment_counts[class_idx]
            print(f"  - {sentiment_name}: {count}/{total_sentiment_samples} mẫu ({count/total_sentiment_samples*100:.2f}%) - weight: {weight:.2f}")
        
        # Topic weights (sử dụng positive_weights cho các nhãn hiếm)
        topic_sums = np.sum(data['train']['topic_labels'], axis=0)
        total_samples = data['train']['topic_labels'].shape[0]
        pos_weights = np.zeros(mappings['num_topics'])
        
        for i in range(mappings['num_topics']):
            # Cân bằng tỷ lệ âm/dương cho mỗi chủ đề
            if topic_sums[i] > 0:
                neg_samples = total_samples - topic_sums[i]
                pos_weights[i] = neg_samples / (topic_sums[i] * 1.0)
            else:
                pos_weights[i] = 10.0  # giá trị mặc định cao nếu không có mẫu dương
        
        # Positive topic samples để tăng cường huấn luyện
        print("Tỷ lệ chủ đề:")
        for i, topic in enumerate(mappings['unique_topics']):
            pos_count = topic_sums[i] 
            print(f"  - {topic}: {pos_count}/{total_samples} mẫu ({pos_count/total_samples*100:.2f}%) - weight: {pos_weights[i]:.2f}")
        
        # Tăng cường dữ liệu cân bằng cho sentiment
        X_train = data['train']['sequences']
        y_train_sentiment = data['train']['sentiment_labels']
        y_train_topic = data['train']['topic_labels']
        
        # Xác định lớp thiểu số và đa số
        minority_class = min(unique_sentiment_counts, key=unique_sentiment_counts.get)
        majority_classes = [i for i in unique_sentiment_counts.keys() if i != minority_class]
        
        # Thêm focal loss cho sentiment để tập trung vào các mẫu khó phân loại
        def focal_loss(gamma=2.0, alpha=0.25):
            def focal_loss_fn(y_true, y_pred):
                # Clip values để tránh log(0)
                y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
                
                # Focal loss formula
                cross_entropy = -y_true * tf.math.log(y_pred)
                weight = tf.pow(1 - y_pred, gamma) * y_true
                
                # Apply weights từ class weights
                weights = np.ones(mappings['num_sentiments'])
                for i in range(mappings['num_sentiments']):
                    if i in sentiment_class_weights:
                        weights[i] = sentiment_class_weights[i]
                
                weight_tensor = tf.constant(weights, dtype=tf.float32)
                weighted_focal_loss = cross_entropy * weight * tf.expand_dims(weight_tensor, 0)
                
                return tf.reduce_sum(weighted_focal_loss, axis=-1)
            return focal_loss_fn
        
        # Weighted binary crossentropy cho topic với positive weights
        def weighted_binary_crossentropy(y_true, y_pred):
            # Áp dụng positive_weights cho từng chủ đề
            weights = tf.constant(pos_weights, dtype=tf.float32)
            y_true = tf.cast(y_true, tf.float32)
            
            # Tránh log(0)
            y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
            
            # Công thức weighted binary crossentropy
            pos_term = -y_true * tf.math.log(y_pred) * weights  # positive samples có trọng số cao hơn
            neg_term = -(1 - y_true) * tf.math.log(1 - y_pred)
            
            return tf.reduce_mean(pos_term + neg_term, axis=-1)
        
        # Biên dịch lại mô hình với các trọng số
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
            loss={
                'sentiment_output': focal_loss(gamma=2.0),  # Dùng focal loss thay vì weighted categorical
                'topic_output': weighted_binary_crossentropy
            },
            loss_weights={
                'sentiment_output': 1.5,  # Tăng trọng số cho sentiment
                'topic_output': 1.0
            },
            metrics={
                'sentiment_output': ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
                'topic_output': [tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
            }
        )
        
        # Callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_sentiment_output_accuracy',
                patience=10,  # Tăng patience để cho mô hình thối gian hội tụ
                restore_best_weights=True,
                mode='max'
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_sentiment_output_accuracy',
                factor=0.5,
                patience=3,
                min_lr=1e-6,
                mode='max'
            ),
            # Thêm checkpoint để lưu mô hình tốt nhất theo sentiment accuracy
            tf.keras.callbacks.ModelCheckpoint(
                filepath='best_sentiment_model.keras',
                save_best_only=True,
                monitor='val_sentiment_output_accuracy',
                mode='max'
            )
        ]
        
        try:
            # Huấn luyện
            history = model.fit(
                data['train']['sequences'],
                {
                    'sentiment_output': data['train']['sentiment_labels'],
                    'topic_output': data['train']['topic_labels']
                },
                validation_data=(
                    data['val']['sequences'],
                    {
                        'sentiment_output': data['val']['sentiment_labels'],
                        'topic_output': data['val']['topic_labels']
                    }
                ),
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=callbacks
                # Đã loại bỏ class_weight vì không hỗ trợ cho mô hình đa đầu ra
            )
            
            # Lưu lịch sử huấn luyện để phân tích sau
            self.history = history.history
            
            # Lưu mô hình
            self.save_model(model, mappings)
            
            return model
        except Exception as e:
            print(f"Lỗi khi huấn luyện mô hình: {str(e)}")
            return None
    
    def predict_sentiment(self, model, text, mappings):
        """Dự đoán tình cảm cho văn bản với các cải tiến"""
        if model is None or mappings is None:
            print("Không có mô hình hoặc ánh xạ nhãn")
            return "Không xác định", 0.0
        
        # Tiền xử lý văn bản
        cleaned_text = self.clean_text(text)
        sequence = self.tokenizer.texts_to_sequences([cleaned_text])
        padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')
        
        # Tạo nhiều biến thể của một câu để tăng cường hiệu suất dự đoán
        # Ví dụ: thêm dấu câu biểu thị cảm xúc, thay đổi từ ngữ nhẹ, v.v.
        variations = [
            padded,  # Câu gốc
        ]
        
        # Dự đoán
        try:
            # Dự đoán tất cả các biến thể
            all_predictions = []
            
            for var in variations:
                predictions = model.predict(var)
                all_predictions.append(predictions[0][0])  # Lấy kết quả sentiment
            
            # Lấy trung bình của tất cả các dự đoán
            avg_predictions = np.mean(all_predictions, axis=0)
            
            # Xử lý kết quả tình cảm
            sentiment_idx = np.argmax(avg_predictions)
            sentiment = mappings['idx_to_sentiment'][sentiment_idx]
            confidence = float(avg_predictions[sentiment_idx])
            
            # Thêm logic để xác định mức độ chắc chắn
            confidence_level = "cao" if confidence > 0.8 else "trung bình" if confidence > 0.6 else "thấp"
            
            return sentiment, confidence, confidence_level
        except Exception as e:
            print(f"Lỗi khi dự đoán: {str(e)}")
            return "Không xác định", 0.0, "không xác định"
    
    def predict_topics(self, model, text, mappings, threshold=None, max_topics=5):
        """Dự đoán chủ đề cho văn bản
        
        Args:
            model: Mô hình đã huấn luyện
            text: Văn bản cần dự đoán
            mappings: Bản đồ ánh xạ nhãn
            threshold: Ngưỡng xác suất để chọn chủ đề (mặc định: 0.2)
            max_topics: Số lượng chủ đề tối đa trả về (mặc định: 5)
            
        Returns:
            Tuple (selected_topics, confidences): Danh sách chủ đề và độ tin cậy tương ứng
        """
        if model is None or mappings is None:
            print("Không có mô hình hoặc ánh xạ nhãn")
            return [], []
        
        if threshold is None:
            threshold = self.topic_threshold
            
        # Tiền xử lý văn bản
        cleaned_text = self.clean_text(text)
        sequence = self.tokenizer.texts_to_sequences([cleaned_text])
        padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')
        
        # Dự đoán
        try:
            predictions = model.predict(padded)
            
            # Xử lý kết quả chủ đề (lấy đầu ra thứ hai là topics)
            topic_preds = predictions[1][0]  # Lấy dự đoán chủ đề
            
            # Tạo danh sách các chủ đề với xác suất
            topics_with_probs = [(mappings['idx_to_topic'][i], float(prob)) 
                             for i, prob in enumerate(topic_preds) 
                             if prob >= threshold]
            
            # Sắp xếp theo độ tin cậy giảm dần và chỉ lấy tối đa max_topics chủ đề
            topics_with_probs.sort(key=lambda x: x[1], reverse=True)
            topics_with_probs = topics_with_probs[:max_topics]
            
            # Tách thành hai danh sách riêng biệt
            if topics_with_probs:
                selected_topics, confidences = zip(*topics_with_probs)
                return list(selected_topics), list(confidences)
            else:
                return [], []
                
        except Exception as e:
            print(f"Lỗi khi dự đoán chủ đề: {str(e)}")
            return [], []
            
    def predict_batch(self, model, texts, mappings, topic_threshold=None, max_topics=5):
        """Dự đoán tình cảm và chủ đề cho một loạt văn bản
        
        Args:
            model: Mô hình đã huấn luyện
            texts: Danh sách các văn bản cần dự đoán
            mappings: Bản đồ ánh xạ nhãn
            topic_threshold: Ngưỡng xác suất để chọn chủ đề (mặc định: 0.2)
            max_topics: Số lượng chủ đề tối đa trả về (mặc định: 5)
            
        Returns:
            Danh sách kết quả dự đoán, mỗi kết quả bao gồm text, sentiment, sentiment_confidence, topics, topic_confidences
        """
        if model is None or mappings is None:
            print("Không có mô hình hoặc ánh xạ nhãn")
            return []
        
        if topic_threshold is None:
            topic_threshold = self.topic_threshold
            
        # Làm sạch và tiền xử lý văn bản
        cleaned_texts = [self.clean_text(text) for text in texts]
        sequences = self.tokenizer.texts_to_sequences(cleaned_texts)
        padded = pad_sequences(sequences, maxlen=MAX_LENGTH, padding='post')
        
        # Dự đoán
        try:
            predictions = model.predict(padded)
            sentiment_preds = predictions[0]  # [batch_size, num_sentiments]
            topic_preds = predictions[1]  # [batch_size, num_topics]
            
            # Xử lý kết quả
            results = []
            for i, (sent_pred, topic_pred) in enumerate(zip(sentiment_preds, topic_preds)):
                # Sentiment
                sentiment_idx = np.argmax(sent_pred)
                sentiment = mappings['idx_to_sentiment'][sentiment_idx]
                sentiment_confidence = float(sent_pred[sentiment_idx])
                confidence_level = "cao" if sentiment_confidence > 0.8 else "trung bình" if sentiment_confidence > 0.6 else "thấp"
                
                # Topics - tạo danh sách với xác suất
                topics_with_probs = [(mappings['idx_to_topic'][j], float(prob)) 
                                 for j, prob in enumerate(topic_pred) 
                                 if prob >= topic_threshold]
                
                # Sắp xếp theo độ tin cậy giảm dần và chỉ lấy tối đa max_topics chủ đề
                topics_with_probs.sort(key=lambda x: x[1], reverse=True)
                topics_with_probs = topics_with_probs[:max_topics]
                
                # Tách thành hai danh sách riêng biệt
                if topics_with_probs:
                    selected_topics, topic_confidences = zip(*topics_with_probs)
                    selected_topics = list(selected_topics)
                    topic_confidences = list(topic_confidences)
                else:
                    selected_topics, topic_confidences = [], []
                
                results.append({
                    'text': texts[i],
                    'sentiment': sentiment,
                    'sentiment_confidence': sentiment_confidence,
                    'confidence_level': confidence_level,
                    'topics': selected_topics,
                    'topic_confidences': topic_confidences
                })
            
            return results
        except Exception as e:
            print(f"Lỗi khi dự đoán hàng loạt: {str(e)}")
            return []
    
    def evaluate(self, model, data, mappings):
        """Đánh giá mô hình trên tập validation với tập trung vào độ chính xác cảm xúc"""
        if model is None or data is None or mappings is None:
            print("Không có mô hình, dữ liệu hoặc ánh xạ nhãn để đánh giá")
            return None
        
        try:
            # Đánh giá
            print("Đánh giá mô hình trên tập validation...")
            results = model.evaluate(
                data['val']['sequences'],
                {
                    'sentiment_output': data['val']['sentiment_labels'],
                    'topic_output': data['val']['topic_labels']
                }
            )
            
            # In kết quả
            metrics = ['loss', 'sentiment_output_loss', 'topic_output_loss', 
                       'sentiment_output_accuracy', 'topic_output_auc', 
                       'sentiment_output_precision', 'sentiment_output_recall', 
                       'topic_output_precision', 'topic_output_recall']
            for i, metric in enumerate(metrics):
                if i < len(results):
                    print(f"{metric}: {results[i]:.4f}")
            
            # Phân tích chi tiết hơn cho sentiment
            print("\nĐánh giá chi tiết sentiment:")
            
            # Lấy dự đoán
            predictions = model.predict(data['val']['sequences'])
            sentiment_preds = predictions[0]
            
            # Chuyển đổi về nhãn số
            true_labels = np.argmax(data['val']['sentiment_labels'], axis=1)
            pred_labels = np.argmax(sentiment_preds, axis=1)
            
            # Tính confusion matrix
            from sklearn.metrics import confusion_matrix, classification_report
            
            cm = confusion_matrix(true_labels, pred_labels)
            print("Confusion Matrix:")
            sentiment_names = [mappings['idx_to_sentiment'][i] for i in range(mappings['num_sentiments'])]
            
            # In confusion matrix dễ đọc
            print("Thực tế \\ Dự đoán:")
            header = "    " + "".join([f"{name[:8]:12s}" for name in sentiment_names])
            print(header)
            
            for i, row in enumerate(cm):
                print(f"{sentiment_names[i][:8]:8s} {' '.join([f'{x:11d}' for x in row])}")
            
            # In classification report
            print("\nClassification Report:")
            report = classification_report(
                true_labels, pred_labels,
                target_names=sentiment_names,
                digits=4
            )
            print(report)
            
            return results
        except Exception as e:
            print(f"Lỗi khi đánh giá mô hình: {str(e)}")
            return None
    
    def save_model(self, model, mappings):
        """Lưu mô hình và ánh xạ nhãn"""
        model_save_path = "vietnamese_sentiment_model.keras"  
        model.save(model_save_path)
        
        # Lưu mappings để sử dụng khi tải mô hình
        mappings_path = "vietnamese_sentiment_mappings.json"
        with open(mappings_path, 'w', encoding='utf-8') as f:
            # Chuyển đổi các key từ int sang str để có thể lưu thành JSON
            serializable_mappings = {
                'unique_sentiments': mappings['unique_sentiments'],
                'sentiment_to_idx': mappings['sentiment_to_idx'],
                'idx_to_sentiment': {str(i): s for i, s in mappings['idx_to_sentiment'].items()},
                'num_sentiments': mappings['num_sentiments'],
                'unique_topics': mappings['unique_topics'],
                'topic_to_idx': mappings['topic_to_idx'],
                'idx_to_topic': {str(i): t for i, t in mappings['idx_to_topic'].items()},
                'num_topics': mappings['num_topics'],
                'vocab_size': mappings['vocab_size']
            }
            json.dump(serializable_mappings, f, ensure_ascii=False, indent=2)
        
        print(f"Đã lưu mô hình tại {model_save_path}")
        print(f"Đã lưu ánh xạ nhãn tại {mappings_path}")
        
    def load_model(self, model_path=None, mappings_path=None):
        """Tải mô hình đã huấn luyện"""
        # Đường dẫn mặc định
        if model_path is None:
            model_path = "vietnamese_sentiment_model.keras"
        if mappings_path is None:
            mappings_path = "vietnamese_sentiment_mappings.json"
            
        try:
            # Kiểm tra tập tin tồn tại
            if not os.path.exists(model_path):
                print(f"Không tìm thấy file mô hình tại {model_path}")
                return None, None
                
            if not os.path.exists(mappings_path):
                print(f"Không tìm thấy file ánh xạ nhãn tại {mappings_path}")
                return None, None
            
            # Tải mappings
            with open(mappings_path, 'r', encoding='utf-8') as f:
                loaded_mappings = json.load(f)
                
            # Chuyển đổi key từ str về int
            if 'idx_to_sentiment' in loaded_mappings:
                loaded_mappings['idx_to_sentiment'] = {int(i): s for i, s in loaded_mappings['idx_to_sentiment'].items()}
            if 'idx_to_topic' in loaded_mappings:
                loaded_mappings['idx_to_topic'] = {int(i): t for i, t in loaded_mappings['idx_to_topic'].items()}
            
            # Tạo các hàm loss tùy chỉnh trống để tải mô hình
            # (chúng sẽ được cập nhật lại khi huấn luyện)
            def custom_sentiment_loss(y_true, y_pred):
                return tf.keras.losses.categorical_crossentropy(y_true, y_pred)
                
            def custom_topic_loss(y_true, y_pred):
                return tf.keras.losses.binary_crossentropy(y_true, y_pred)
                
            # Tải mô hình
            model = tf.keras.models.load_model(
                model_path,
                custom_objects={
                    'weighted_categorical_crossentropy': custom_sentiment_loss,
                    'weighted_binary_crossentropy': custom_topic_loss
                }
            )
            
            print(f"Đã tải mô hình từ {model_path}")
            return model, loaded_mappings
            
        except Exception as e:
            print(f"Lỗi khi tải mô hình: {str(e)}")
            return None, None

# Ví dụ sử dụng
if __name__ == "__main__":
    try:
        # Khởi tạo classifier
        classifier = VietnameseTextClassifier()
        
        # Tải dữ liệu huấn luyện JSON
        json_path = "/kaggle/input/datata/train data.json"  # Đường dẫn đến file JSON
        
        # Kiểm tra file JSON tồn tại
        if not os.path.exists(json_path):
            print(f"Không tìm thấy file dữ liệu tại {json_path}")
            # Thử tải mô hình đã huấn luyện
            model, mappings = classifier.load_model()
        else:
            train_df = classifier.load_train_data(json_path)
            
            # Chuẩn bị dữ liệu (tự động tạo tập validation từ dữ liệu huấn luyện)
            data, mappings = classifier.prepare_data(train_df)
            
            # Huấn luyện mô hình
            model = classifier.train(data, mappings)
            
            # Đánh giá mô hình
            if model is not None:
                classifier.evaluate(model, data, mappings)
        
        # Ví dụ dự đoán
        example_texts = [
            "Tôi rất thích sản phẩm này, chất lượng tuyệt vời!",
            "Dịch vụ tại cửa hàng này thật tệ.",
            "Sản phẩm đúng như mô tả, giao hàng đúng hẹn."
        ]
        
        # Dự đoán dùng mô hình hiện tại
        if model is not None and mappings is not None:
            print("\nDự đoán đơn:")
            for text in example_texts:
                sentiment, confidence, confidence_level = classifier.predict_sentiment(model, text, mappings)
                topics, topic_conf = classifier.predict_topics(model, text, mappings)
                
                print(f"Văn bản: {text}")
                print(f"Tình cảm: {sentiment}, Độ tin cậy: {confidence:.4f}, Mức độ tin cậy: {confidence_level}")
                print(f"Chủ đề: {topics}")
                print(f"Độ tin cậy chủ đề: {[f'{conf:.4f}' for conf in topic_conf]}")
                print("-" * 50)
            
            print("\nDự đoán hàng loạt:")
            batch_results = classifier.predict_batch(model, example_texts, mappings)
            for result in batch_results:
                print(f"Văn bản: {result['text']}")
                print(f"Tình cảm: {result['sentiment']}, Độ tin cậy: {result['sentiment_confidence']:.4f}, Mức độ: {result['confidence_level']}")
                print(f"Chủ đề: {result['topics']}")
                print(f"Độ tin cậy chủ đề: {[f'{conf:.4f}' for conf in result['topic_confidences']]}")
                print("-" * 50)
        else:
            print("Không thể dự đoán do không có mô hình hoặc ánh xạ nhãn")
        
    except Exception as e:
        print(f"Lỗi: {str(e)}")


Khởi tạo Vietnamese Text Classifier với mô hình BiLSTM...
Đang tải dữ liệu huấn luyện từ /kaggle/input/datata/train data.json...
Cấu trúc dữ liệu: <class 'list'>
Số lượng mục: 1696
Phát hiện định dạng Label Studio
Đã tải 1695 mẫu dữ liệu huấn luyện
Tìm thấy 23 chủ đề khác nhau
Đã tạo thêm 1197 mẫu tăng cường để cân bằng dữ liệu cảm xúc
Chia dữ liệu: 2313 mẫu huấn luyện, 579 mẫu validation
Huấn luyện mô hình...
Xây dựng mô hình BiLSTM kép cho phân loại tình cảm và chủ đề...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Tỷ lệ cảm xúc:
  - Trung tính: 776/2313 mẫu (33.55%) - weight: 0.99
  - Tiêu cực: 763/2313 mẫu (32.99%) - weight: 1.01
  - Tích cực: 774/2313 mẫu (33.46%) - weight: 1.00
Tỷ lệ chủ đề:
  - Bất động sản: 86.0/2313 mẫu (3.72%) - weight: 25.90
  - Chính trị: 84.0/2313 mẫu (3.63%) - weight: 26.54
  - Chứng khoán: 43.0/2313 mẫu (1.86%) - weight: 52.79
  - Covid-19: 292.0/2313 mẫu (12.62%) - weight: 6.92
  - Công nghệ: 154.0/2313 mẫu (6.66%) - weight: 14.02
  - Du lịch: 56.0/2313 mẫu (2.42%) - weight: 40.30
  - Game: 24.0/2313 mẫu (1.04%) - weight: 95.38
  - Giao thông: 44.0/2313 mẫu (1.90%) - weight: 51.57
  - Giáo dục: 61.0/2313 mẫu (2.64%) - weight: 36.92
  - Giải trí: 292.0/2313 mẫu (12.62%) - weight: 6.92
  - Hóng biến: 257.0/2313 mẫu (11.11%) - weight: 8.00
  - Khoa học: 33.0/2313 mẫu (1.43%) - weight: 69.09
  - Không xác định: 24.0/2313 mẫu (1.04%) - weight: 95.38
  - Kinh tế: 405.0/2313 mẫu (17.51%) - weight: 4.71
  - Môi trường: 102.0/2313 mẫu (4.41%) - weight: 21.68
  - Phim ảnh: 54

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:678: UserWarning: Gradients do not exist for variables ['attention_weight', 'attention_bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


73/73 ━━━━━━━━━━━━━━━━━━━━ 16s 58ms/step - loss: 2.0238 - sentiment_output_accuracy: 0.3341 - sentiment_output_loss: 0.7348 - sentiment_output_precision: 0.0000e+00 - sentiment_output_recall: 0.0000e+00 - topic_output_auc: 0.5098 - topic_output_loss: 1.2891 - topic_output_precision_1: 0.0795 - topic_output_recall_1: 0.3941 - val_loss: 2.0015 - val_sentiment_output_accuracy: 0.5492 - val_sentiment_output_loss: 0.6998 - val_sentiment_output_precision: 0.0000e+00 - val_sentiment_output_recall: 0.0000e+00 - val_topic_output_auc: 0.4919 - val_topic_output_loss: 1.2826 - val_topic_output_precision_1: 0.0813 - val_topic_output_recall_1: 0.3691 - learning_rate: 0.0010
Epoch 2/15
73/73 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: 1.8208 - sentiment_output_accuracy: 0.5937 - sentiment_output_loss: 0.5440 - sentiment_output_precision: 0.6437 - sentiment_output_recall: 0.2484 - topic_output_auc: 0.5425 - topic_output_loss: 1.2767 - topic_output_precision_1: 0.0837 - topic_output_recall_1: 0.4392 - va